# Deploying Iris Classifier to Clipper cluster

In this example, we will train an classifier with SK learn and deploy it to local clipper cluster.

### Import required packages

In [ ]:
!pip install bentoml
!pip install pandas sklearn
!pip install clipper_admin

### Train iris classifier

In [ ]:
from sklearn import svm
from sklearn import datasets

clf = svm.SVC()
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)

### Save trained model as BentoML service archive

In [ ]:
%%writefile iris_classifier.py
from bentoml import BentoService, api, env, artifacts
from bentoml.artifact import PickleArtifact
from bentoml.handlers import DataframeHandler

# You can also import your own python module here and BentoML will automatically
# figure out the dependency chain and package all those python modules

@artifacts([PickleArtifact('model')])
@env(conda_pip_dependencies=["scikit-learn"])
class IrisClassifier(BentoService):

    @api(DataframeHandler)
    def predict(self, df):
        # arbitrary preprocessing or feature fetching code can be placed here 
        return self.artifacts.model.predict(df)

In [ ]:
# 1) import the custom BentoService defined above
from iris_classifier import IrisClassifier

# 2) `pack` it with required artifacts
svc = IrisClassifier.pack(model=clf)

# 3) save packed BentoService as archive
saved_path = svc.save('/tmp/bentoml_archive')

# archive will be saved to:
print(saved_path)

# Deploy to Clipper cluster

In [ ]:
from clipper_admin import ClipperConnection, DockerContainerManager

# Create clipper cluster connection
clipper_conn = ClipperConnection(DockerContainerManager())

# BentoML will deploy the BentoService archive as model to the clipper cluster
model_name, model_version = deploy_bentoml(clipper_conn, saved_path, api_name)

app_name = 'iris-classifier'

# Register a new application on the clipper cluster.  Application is an REST endpoint that expose
# to outside of the cluster.
app = clipper_conn.register_application(
    name=app_name,
    input_type='strings',
    default_output='default result',
    slo_micros=100000
)

# Linking the deployed model with registered application on clipper cluster
linked = clipper_conn.link_model_to_app(app_name, model_name)


### Make request to iris-classifier app

We are going to make a simple POST request to the clipper cluster and getting back prediction result.

In [ ]:
import json
import requests
from datetime import datetime

url = 'http://%s/test_app/predict' % clipper_conn.get_query_addr()

x = '[[5.0, 4.3, 1.2, 4.9]]'
req_json = json.dumps({'input': x})
headers = {'Content-type': 'application/json'}
print('Requesting prediction to clipper cluster')
start = datetime.now()
r = requests.post(url, headers=headers, data=req_json)
end = datetime.now()
latency = (end - start).total_seconds() * 1000.0
print("'%s', %f ms" % (r.text, latency))

### Clean up Clipper deployment

In [ ]:
clipper_conn.stop_all()